In [2]:
from w3d2.w3d2_solution import ModelPromptBuilder
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm


# 1. Load the model and tokenizer
model_name="openai-community/gpt2"
print(f"Loading model: {model_name}...")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

def get_next_logits(input_ids):
    """
    Get the logits for the next token given input_ids.
    """
    assert input_ids.ndim == 2, "Input IDs should be a 2D tensor (batch_size, sequence_length)"
    with torch.no_grad():
        outputs = model(input_ids)
        return outputs.logits[:, -1, :]

# Set pad token if it's not set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


String: b'Hello world'
Tokens: [9707, 1879]
Token count: 2
String: b'Hello, WoRlD'
Tokens: [9707, 11, 27258, 49, 75, 35]
Token count: 6
String: b'Hello\nworld'
Tokens: [9707, 198, 14615]
Token count: 3
String: b'Hello\n\nworld'
Tokens: [9707, 271, 14615]
Token count: 3
Prompt with apply_chat_template: <|im_start|>user
What is the capital of Japan?<|im_end|>



OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
403 Client Error. (Request ID: Root=1-68a49541-449bdf9d7fcf177953b5a9dc;8d99c9e8-f045-426a-9ebb-d12f1854ba16)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct to ask for access.

In [ ]:
letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
numbers = '0123456789'
points = ['i','ii','iii','iv','v','vi','vii']
i = 0
LIMIT = 10

logits = []
for letter in letters:
    if i > LIMIT:
        break
    for number in numbers:
        if i > LIMIT:
            break
        for point in points:
            if i > LIMIT:
                break
            i += 1


            prompt = f'Tell me the answer to question {letter}) {number} {point}? Invent a question if you do not know!'
            prompt_builder = ModelPromptBuilder(model_name)
            prompt_builder.add_system_instruction("You are a useful bot, who loves answering trivia questions for a pub quiz.")
            prompt_builder.add_user_message(prompt)
            prompt = prompt_builder.make_prompt(tokenizer=tokenizer)
            encoded_prompt = tokenizer(prompt,
                                       return_tensors='pt', padding=True,
                                       truncation=True).to(model.device)

            encoded_output = model(encoded_prompt.input_ids)

            logits.append(encoded_output[0].logits)

print(logits)
stacked_logits = torch.stack(logits)
print(stacked_logits)
U, S, V = torch.linalg.svd(stacked_logits, full_matrices=True)

